<div class="alert alert-danger">
**Due date:** 2017-02-03
</div>

# Lab 2: Language Modelling

**Students:** Ludvig Noring (ludno249), Michael Sörsäter (micso554), Victor Tranell (victr593)

## Introduction

In this lab you will experiment with $n$-gram models. You will test various parameters that influence these models&rsquo; quality and train to estimate models with additive smoothing.

The following lines of code import the Python modules needed for this lab:

In [1]:
import nlp2
import ngrams

The data for this lab consists of Arthur Conan Doyle&rsquo;s novels about Sherlock Holmes: *The Adventures of Sherlock Holmes*, *The Memoirs of Sherlock Holmes*, *The Return of Sherlock Holmes*, *His Last Bow* and *The Case-Book of Sherlock Holmes*. The next piece of code loads the first three of these as training data:

In [2]:
training_data = nlp2.read_data("/home/TDDE09/labs/nlp2/data/advs.txt",
                               "/home/TDDE09/labs/nlp2/data/mems.txt",
                               "/home/TDDE09/labs/nlp2/data/retn.txt")

The data is represented as a list of sentences, where one sentence is represented as a list of tokens (strings). The next line prints the 101th sentence:

In [3]:
print(training_data[100])

["'", 'Let', 'us', 'glance', 'at', 'our', 'Continental', 'Gazetteer', '.']


## Relation between a model’s quality and its order

In the first part of this lab you will examine the relation between an $n$-gram model’s quality and its **order**, i.e. the value of&nbsp;$n$. You will do both a qualitative and quantitative evaluation with the help of the entropy measure.

### Qualitative evaluation

The following line trains a bigram-model of the class `ngrams.Model` on the training data.

In [4]:
model = nlp2.train(ngrams.Model, 2, training_data)

With this model you are able to generate random sentences. Every time you run the following code cell a new sentence is generated.

In [5]:
print(" ".join(model.generate()))

" As I came the money in which he see your coffee .


Look at the sentences. Do they sound natural?

<div class="panel panel-primary">
<div class="panel-heading">Problem 1</div>
<div class="panel-body">
Train a unigram-, bigram-, trigram-, and quadrigram-model, and generate random sentences with each. How does the quality of the sentences change with the model’s order? Explain your observations using your understanding from how an $n$-gram model works. Use some generated sentences in order to illustrate your discussion. How would the sentences look like for higher values of $n$, such as $n=10$?
</div>
</div>

In [6]:
models = []
modelNames = ["Uni", "Bi", "Tri", "Quadri"]
for i, m in enumerate(modelNames):
    model = nlp2.train(ngrams.Model, i + 1, training_data)
    models.append(model)
    
    print("Traing", m + "gram:")
    
    print("Ex1:", " ".join(model.generate()))
    print("Ex2:", " ".join(model.generate()))
    
    print()
    
    
model_10 = nlp2.train(ngrams.Model, 10, training_data)
print("Traing 10-gram model")
print("Ex1:", " ".join(model_10.generate()))
print("Ex2:", " ".join(model_10.generate()))


Traing Unigram:
Ex1: endeavour the at , believe
Ex2: you roamed a was it to , . Quick upon gallop into man's that " yet and it which Lord I paralyzing of This

Traing Bigram:
Ex1: " Fire !
Ex2: Then he mounted the queerest club , but is what exact science by trying ones .

Traing Trigram:
Ex1: Such a charge of conspiracy , as we emerged from the boggy portion of the day in the hall and into his private room , and it was him or he might throw a fiercer light into anything quite so much annoyance upon her husband and child only just returned upon the varnish would have nothing to hide his own bedroom , and a little mystification , Lestrade , grimly .
Ex2: " Can I speak took place .

Traing Quadrigram:
Ex1: What a charming , old-fashioned room !
Ex2: The older man seemed numbed and dazed with a heavy brassy Albert chain , and the gentleman was about to renew her entreaties when a door slammed overhead , and the sun was just grazing the top of a harmonium beside the door .

Traing 10-gram 

When n = 1 we simply generate sentences using a "bag of words-model". So that could result in the word "I" occuring 2 times in a row. Upping the n to 2 that likelihood of that happening is much smaller since I never occurs after I in the sherlock holmes texts. When n grows too large however, say 10, that exact sequence of 9 tokens only occurs one or two times so the model will overfit. *EOS*

### Quantitative evaluation

In order to do a quantitative evaluation of a model we can compute its **entropy** on held-out data. We will use the first part of the novel *The Adventures of Sherlock Holmes* for this. It is loaded by the following command:

In [7]:
test_data = nlp2.read_data("/home/TDDE09/labs/nlp2/data/test.txt")

The next piece of code trains a bigram-model and computes its entropy on the test data:

In [8]:
model = nlp2.train(ngrams.Model, 2, training_data)
print("Entropy:", nlp2.evaluate(model, test_data))

Entropy: 3.426862596420277


<div class="panel panel-primary">
<div class="panel-heading">Problem 2</div>
<div class="panel-body">
Compute the entropy for the four models you created for the previous problem. How does the model’s entropy change with the model’s order? Explain using your knowledge of the entropy measure.
</div>
</div>

In [9]:
for i, m in enumerate(modelNames):
    print(m + "gram model.")
    print("Entropy: {0:.3f}".format(nlp2.evaluate(models[i], test_data)))
    print()

Unigram model.
Entropy: 7.338

Bigram model.
Entropy: 3.427

Trigram model.
Entropy: 1.429

Quadrigram model.
Entropy: 0.544



As the order increases, the entropy decreases.

With a low n-value model, the number of possible outcomes is high and therefore the entropy is high.

## Relation between a model’s quality and the estimation method

In the second part of this lab you will implement and evaluate various estimation methods. In order to do that you will need to know how the lab system is built up.

### The content of a model

When you call the `train()` function (like you did above), the system creates an $n$-gram model of the given class (so far: `ngrams.Model`) and with the given order (the value of $n$) and trains the model on the given data set. For the second part of this lab you will use your own model class. We start with defining the class in such a way that it simply calls the corresponding methods of the superclass:

In [10]:
class Model(ngrams.Model):
    
    def order(self):
        """Return the order of this model (an integer)."""
        return super().order()
    
    def vocabulary(self):
        """Return this model's vocabulary (a set)."""
        return super().vocabulary()
    
    def freq(self, ctxt, word):
        """Return the number of occurrences of `word` (a string) after `ctxt` (a tuple of strings)."""
        return super().freq(ctxt, word)
    
    def total(self, ctxt):
        """Return the total number of ngrams that start with `ctxt` (a tuple of strings)."""
        return super().total(ctxt)
    
    def prob(self, ctxt, word):
        """Return the probability for `word` (a string) given `ctxt` (a tuple of strings)."""
        return super().prob(ctxt, word)

The next piece of code trains a bigram-model of the class `Model` and prints the model’s order (an integer) and the size of its vocabulary (a set of strings, represented by Python’s `set` type).

In [11]:
model = nlp2.train(Model, 2, training_data)
print("order of the model:", model.order())
print("number of words in the model's vocabulary:", len(model.vocabulary()))

order of the model: 2
number of words in the model's vocabulary: 15339


#### Look up an n-gram’s absolute frequency

A trained model consists primarily of a table with absolute frequencies for all $n$-grams that appear in the text it was trained on. In order to look up an $n$-gram’s absolute frequency one can use the method `freq()`. An $n$-gram is divided into two parts: an $(n-1)$-gram called **context** (`ctxt`) and a final unigram (`word`). In Python the context is represented as a tuple of strings and the unigram as a normal string.

If you want to train a trigram model and then know the absolute frequency for the trigram *Mr. Sherlock Holmes* you can write:

In [12]:
model = nlp2.train(Model, 3, training_data)
print(model.freq(("Mr.", "Sherlock"), "Holmes"))

50


For training a bigram model and looking up the absolute frequency for the bigram *Baker Street* you can write the following. Note that the context of a bigram model is a 1-tuple of strings, which has a special notation in Python.

In [13]:
model = nlp2.train(Model, 2, training_data)
print(model.freq(("Baker",), "Street"))

67


#### Look up the absolute frequency of an n-gram with a given context

The method `total()` returns the absolute frequency of $n$-grams with the given context. Here is an example for a trigram model:

In [14]:
model = nlp2.train(Model, 3, training_data)
print(model.total(("Mr.", "Sherlock")))

50


<div class="panel panel-primary">
<div class="panel-heading">Problem 3</div>
<div class="panel-body">
Train a bigram model and use it to calculate the following values, using the methods shown above.
</div>
</div>

In [15]:
model = nlp2.train(Model, 2, training_data)

**3.1.** the absolute frequency for the bigram *Sherlock Holmes*

In [16]:
print(model.freq(("Sherlock",), "Holmes"))

195


**3.2.** the absolute frequency of bigrams with the context *Sherlock*

In [17]:
print(model.total(("Sherlock",)))

210


**3.3.** the absolute frequency for the unigram *Sherlock*

In [18]:
model_uni = nlp2.train(Model, 1, training_data)
print(model_uni.freq((), "Sherlock"))

210


**3.4.** the number of words in the vocabulary

In [19]:
print("number of words in the model's vocabulary:", len(model.vocabulary()))

number of words in the model's vocabulary: 15339


**3.5.** a list with all the words following the context *Sherlock*

In [20]:
print(*[word for word in model.vocabulary() if model.prob(("Sherlock",), word) > 0])

has Holmes looked ? everywhere ! , . Holmes's


(For the last exercise you will need to write a bit more than a simple function call.)

### Estimate probabilities with the Maximum Likelihood method

The method `prob()` returns the estimated conditional probability $P(w|c)$ for a word $w$ given a context $c$. The following code snippet trains a trigram model and estimates the pobability for *Holmes* given the context *Mr. Sherlock*:

In [21]:
model = nlp2.train(Model, 3, training_data)
print(model.prob(("Mr.", "Sherlock"), "Holmes"))

1.0


(What does the returned value imply?)

<div class="panel panel-primary">
<div class="panel-heading">Problem 4</div>
<div class="panel-body">
Do your own implementation of the method `prob()`. The method should estimate probabilities using the Maximum Likelihood method. Test your implementation by redoing Exercise&nbsp;2 with the new class; you should get the same result as before. Use the code you wrote in Exercise&nbsp;3 in order to solve the exercise.
</div>
</div>

In [22]:
class Model(ngrams.Model):
    
    def prob(self, ctxt, word):
        """Return the probability for `word` (a string) given `ctxt` (a tuple of strings)."""
        
        return self.freq(ctxt, word) / self.total(ctxt)


print("Trigram model")
model = nlp2.train(Model, 3, training_data)
print(model.prob(("Sherlock","Holmes"), "I"))

print()

print("Bigram model")
model = nlp2.train(Model, 2, training_data)
print(model.prob(("Sherlock",), "Holmes"))

Trigram model
0.005128205128205128

Bigram model
0.9285714285714286


In order to solve this exercise you will need to turn the formula for Maximum Likelihood estimation into code. We illustrate the formula for a bigram model. If we write $f(w_1w_2)$ for the number of occurrences of the bigram  $w_1w_2$ and $f(w_1)$ for the number of occurrences of the unigram $w_1$, then the probability for observing $w_2$ given $w_1$ is
$$
P(w_2|w_1) = \frac{f(w_1w_2)}{f(w_1)}
$$

### Problems with Maximum Likelihood estimation

The file `yoda.txt` contains the same text as `test.txt`, but in the jumbled [Yoda-language]( http://itre.cis.upenn.edu/~myl/languagelog/archives/002173.html).

In [23]:
yoda_data = nlp2.read_data("/home/TDDE09/labs/nlp2/data/yoda.txt")

<div class="panel panel-primary">
<div class="panel-heading">Problem 5</div>
<div class="panel-body">
Redo the evaluation of the four previous models with `yoda.txt` as test data. Something unexpected happens for models with $n>1$. Why? Explain the problem with your knowledge of Maximum Likelihood estimation.
</div>
</div>

In [24]:
for i, m in enumerate(modelNames):
    print(m + "gram model.")
    try:
        print("Entropy: {0:.3f}".format(nlp2.evaluate(models[i], yoda_data)))
    except:
        print("I crashed")
        break
    print()

Unigram model.
Entropy: 7.244

Bigram model.
I crashed


Since the yoda data has scrambled the order of the bigrams has been changed. Thus there exists bigrams in the yoda text that does not exist in the training data.

### Estimate probabilities with additive smoothing

For the next problem you are going to do Maximum Likelihood estimation, but with additive smoothing.

<div class="panel panel-primary">
<div class="panel-heading">Problem 6</div>
<div class="panel-body">
<p>
Write a new implementation of the method `prob()`, such that it estimates probabilities with additive smoothing.</p>
<p>
Evaluate the system with new new class using the entropy measure from Problem&nbsp;2. Choose the following values for the smoothing constant $k$:&nbsp;0,00,&nbsp;1,00, 0,10, 0,01. For $k=0$ you should get the same result as in Problem&nbsp;5.
</p>
<p>
How and why does the smoothing constant influence the model’s entropy? Connect to the distribution of the probability mass between observed and imaginary occurrences.
</p>
</div>
</div>

In [25]:
class Model(ngrams.Model):
    
    def prob(self, ctxt, word):
        """Return the probability for `word` (a string) given `ctxt` (a tuple of strings)."""
        
        # Some words in the context isn't seen before
        for ct in ctxt:
            if not ct in self.vocabulary():
                return 1 / len(self.vocabulary())
        # Word is not seen   
        if not word in self.vocabulary():
            return (self.k) / (self.total(ctxt) + self.k * len(self.vocabulary()))
            
                
        return (self.freq(ctxt, word) + self.k) / (self.total(ctxt) + self.k * len(self.vocabulary()))


In [26]:
k_list = [0, 1, 0.10, 0.01]
for i, m in enumerate(modelNames):
    print(m + "gram")
    for k in k_list:
        model = nlp2.train(Model, i + 1, training_data)    
        model.k = k
        try:
            entropy = nlp2.evaluate(model, yoda_data)
        except:
            entropy = 0
        print('k={k}, entropy={entropy:.2f}'.format(**locals()))
    print()

Unigram
k=0, entropy=7.24
k=1, entropy=7.18
k=0.1, entropy=7.24
k=0.01, entropy=7.24

Bigram
k=0, entropy=0.00
k=1, entropy=7.53
k=0.1, entropy=6.27
k=0.01, entropy=5.27

Trigram
k=0, entropy=0.00
k=1, entropy=8.76
k=0.1, entropy=7.38
k=0.01, entropy=5.82

Quadrigram
k=0, entropy=0.00
k=1, entropy=9.05
k=0.1, entropy=7.89
k=0.01, entropy=6.42



Fill the table below with your entropy measures.

<table>
<tr><td></td><td>k = 0.00</td><td>k = 1.00</td><td>k = 0.10</td><td>k = 0.01</td></tr>
<tr><td>n = 1</td><td>7.24</td><td>7.18</td><td>7.24</td><td>7.24</td></tr>
<tr><td>n = 2</td><td>-</td><td>7.53</td><td>6.27</td><td>5.27</td></tr>
<tr><td>n = 3</td><td>-</td><td>8.76</td><td>7.38</td><td>5.82</td></tr>
<tr><td>n = 4</td><td>-</td><td>9.05</td><td>7.89</td><td>6.42</td></tr>
</table>

With k = 1, we smooth the models too much. 
With the bigram model and k = 0.01 we get the lowest entropy which means it predicts the test data well.

### An unseen test set

Your last exercise is to redo the evaluation on a previously unseen test set, texts from the collection *His Last Bow*.

In [27]:
unseen_data = nlp2.read_data("/home/TDDE09/labs/nlp2/data/lstb.txt")

<div class="panel panel-primary">
<div class="panel-heading">Problem 7</div>
<div class="panel-body">
Redo the evaluation from Problem 6 with the new test data. Explain what happens given the differences between `test.txt` and `lstb.txt`.
</div>
</div>

In [28]:
k_list = [0, 1, 0.10, 0.01]

for i, m in enumerate(modelNames):
    print(m + "gram")
    for k in k_list:
        model = nlp2.train(Model, i + 1, training_data)    
        model.k = k
        try:
            entropy = nlp2.evaluate(model, unseen_data)
        except:
            entropy = 0
        print('k={k}, entropy={entropy:.2f}'.format(**locals()))
    print()


Unigram
k=0, entropy=0.00
k=1, entropy=6.26
k=0.1, entropy=6.33
k=0.01, entropy=6.40

Bigram
k=0, entropy=0.00
k=1, entropy=6.83
k=0.1, entropy=5.93
k=0.01, entropy=5.55

Trigram
k=0, entropy=0.00
k=1, entropy=8.42
k=0.1, entropy=7.70
k=0.01, entropy=7.13

Quadrigram
k=0, entropy=0.00
k=1, entropy=8.86
k=0.1, entropy=8.48
k=0.01, entropy=8.16



The unseen data gets higher entropies than the previously seen data for obvious reasons.